# Example on the MNIST Fashion Dataset

### 1. Import of the modules and the dataset

In [1]:
from neural_network import *
from keras.datasets import fashion_mnist

2023-08-31 23:29:42.973116: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 2. Load the MNIST dataset

In [2]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

### 3. Build the neural network

In [3]:
net = NeuralNetwork(
    Normalization(samples=X_train),
    Reshape(output_shape=(1, 28, 28)),
    Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2),
    BatchNorm2d(6),
    Tanh(),
    AvgPool2d(kernel_size=(2, 2), stride=(2, 2)),
    Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
    BatchNorm2d(16),
    Tanh(),
    AvgPool2d(kernel_size=(2, 2), stride=(2, 2)),
    Flatten(),
    Linear(in_features=400, out_features=120),
    Tanh(),
    Linear(in_features=120, out_features=84),
    Tanh(),
    Linear(in_features=84, out_features=10),
    SoftmaxCategoricalCrossEntropy()
)
print(net)

NeuralNetwork:
 (0) Normalization(norm=255.0, dtype=float32)
 (1) Reshape(output_shape=(1, 28, 28))
 (2) Conv2d(in_channels=1, out_channels=6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (3) BatchNorm2d(6, eps=1e-05, momentum=0.1)
 (4) Tanh()
 (5) AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
 (6) Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5), stride=(1, 1), padding=(0, 0), optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (7) BatchNorm2d(16, eps=1e-05, momentum=0.1)
 (8) Tanh()
 (9) AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
 (10) Flatten()
 (11) Linear(in_features=400, out_features=120, optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (12) Tanh()
 (13) Linear(in_features=120, out_features=84, optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), ini

### 4. Choose the number of samples to train on

In [4]:
# Select N samples to train on:
N = 5000
X_train, y_train = X_train[:N], y_train[:N]
X_train.shape, y_train.shape

((5000, 28, 28), (5000,))

### 5. Train the neural network

In [5]:
net.fit(X_train, y_train, epochs=5, batch_size=64, shuffle=True)

Training on 5000 samples:
Epoch    1 of 5    	 Average Error = 0.843117 	 Average Accuracy = 70.46%                                   
Epoch    2 of 5    	 Average Error = 0.519771 	 Average Accuracy = 81.40%                                   
Epoch    3 of 5    	 Average Error = 0.447029 	 Average Accuracy = 84.06%                                   
Epoch    4 of 5    	 Average Error = 0.403950 	 Average Accuracy = 85.74%                                   
Epoch    5 of 5    	 Average Error = 0.362858 	 Average Accuracy = 87.34%                                   
Training time : 00 hours, 01 minutes, 06 seconds


### 6. Make predictions on the test samples

In [6]:
# Prediction on the 10,000 test samples:
y_pred = net.predict(X_test, to="labels")
# Shows the first 10 predicted labels vs the true labels:
y_pred[:10], y_test[:10]

(array([9, 2, 1, 1, 6, 1, 4, 6, 7, 7]),
 array([9, 2, 1, 1, 6, 1, 4, 6, 5, 7], dtype=uint8))

### 7. Evaluate the accuracy score

In [7]:
print(f"Accuracy score on the test set: {accuracy_score(y_test, y_pred):.2%}")

Accuracy score on the test set: 83.87%


### 8. Build a confusion matrix

In [8]:
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion matrix:\n{cm}")

Confusion matrix:
[[821   8  14  28   3   2 100   0  24   0]
 [  3 964   3  21   4   0   3   0   2   0]
 [ 15   6 707  11 145   0  97   1  18   0]
 [ 32  21   7 860  30   0  43   0   7   0]
 [  1   3  85  39 783   1  78   0  10   0]
 [  1   0   0   1   0 922   0  47   4  25]
 [157   2 107  31 138   1 526   0  38   0]
 [  0   0   0   0   0  37   0 918   1  44]
 [  1   2   2   7   2   4  15   7 958   2]
 [  0   0   0   0   0  24   1  46   1 928]]


### 9. Displays a classification report

In [9]:
class_labels = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
print(classification_report(cm, class_labels=class_labels, formatted=True))

┌───────────────────┬───────────┬───────────┬───────────┬───────────┐
│       Class       │ Precision │   Recall  │  F1-Score │  Support  │
├───────────────────┼───────────┼───────────┼───────────┼───────────┤
│    T-shirt/top    │  79.63%   │  82.10%   │  80.85%   │   1000    │
│      Trouser      │  95.83%   │  96.40%   │  96.11%   │   1000    │
│     Pullover      │  76.43%   │  70.70%   │  73.45%   │   1000    │
│       Dress       │  86.17%   │  86.00%   │  86.09%   │   1000    │
│       Coat        │  70.86%   │  78.30%   │  74.39%   │   1000    │
│      Sandal       │  93.04%   │  92.20%   │  92.62%   │   1000    │
│       Shirt       │  60.95%   │  52.60%   │  56.47%   │   1000    │
│      Sneaker      │  90.09%   │  91.80%   │  90.94%   │   1000    │
│        Bag        │  90.12%   │  95.80%   │  92.87%   │   1000    │
│    Ankle boot     │  92.89%   │  92.80%   │  92.85%   │   1000    │
└───────────────────┴───────────┴───────────┴───────────┴───────────┘
